In [17]:
import os
os.environ['HF_HOME'] = 'your-cache-dir'

In [18]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# load model: machinelearnear/Phi-3-mini-4k-instruct-milei-gpt, machinelearnear/llama-3-8b-milei-gpt
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "machinelearnear/Phi-3-medium-4k-instruct-milei-gpt", 
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

adapter_config.json:   0%|          | 0.00/737 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/623 [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Mistral patching release 2024.5
   \\   /|    GPU: NVIDIA A10G. Max memory: 22.191 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.26.post1. FA = True.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors.index.json:   0%|          | 0.00/29.9k [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


adapter_model.safetensors:   0%|          | 0.00/262M [00:00<?, ?B/s]

Unsloth 2024.5 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


In [22]:
# function to read yaml file
def read_yaml(config_path):
    import yaml
    with open(config_path, 'r') as file:
        return yaml.safe_load(file)
    
config = read_yaml("../scripts/phi_3_mini_4k_unsloth_finetuning.yaml")
system_message = config.get('system_message', 'You are a nice bot!')

In [25]:
messages = [
    {
        "content": system_message,
        "role": "system",
    },
    {
        "content": "Las peatonales deberian ser privadas o no?",
        "role": "user"
    },
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(
    input_ids = inputs,
    streamer = text_streamer,
    max_new_tokens = 1024,
    use_cache = True,
    temperature=0.1, # creativity and randomness of the response
    top_p=0.9, # dynamically adjusts the number of choices for each predicted token, which helps to maintain diversity and generate more fluent and natural-sounding text
    top_k=50, # limits the number of choices for the next predicted word or token, which helps to speed up the generation process and can improve the quality of the generated text
    repetition_penalty=1.2, # reduce the likelihood of repeating prompt text or getting stuck in a loop
)

<|user|> Las peatonales deberian ser privadas o no?<|end|><|assistant|> No, las peatonales son un derecho.  Yo creo que la gente tiene el derecho a caminar por donde quiera y en cualquier momento del día sin tener miedo ni nada como esas cosas porque yo soy una persona muy liberal con los temas sociales pero también tengo mucha preocupación sobre seguridad pública así que si vos querés hacer algo para mejorar tu vida tenemos que pensarlo desde todas estas perspectivas distintas<|endoftext|>
